# Chapter 2 - Primer on Probability Modeling

The notebook is a code companion to chapter 2 of the book [Causal Machine Learning](https://www.manning.com/books/causal-machine-learning) by Robert Osazuwa Ness.

In [1]:
# For stability of this notebook, I install the versions of pgmpy and pyro that were current at the time of writing.
!pip install pgmpy==0.1.24
!pip install pyro-ppl==1.8.6

## Implementing a discrete distribution table in pgmpy


In [2]:
import pgmpy
from pgmpy.factors.discrete import DiscreteFactor
dist = DiscreteFactor(
    variables=["X"],    #A
    cardinality=[3],    #B
    values=[.45, .30, .25],    #C
    state_names= {'X': ['1', '2', '3']}    #D
)

#A A list of the names of the variables in the factor
#B The cardinality (number of possible outcomes of each variable in the factor
#C The values each variable in the factor can take
#D Dictionary where the key is the variable name and the value is a list of the names of that variable’s outcomes

print(dist)

+------+----------+
| X    |   phi(X) |
+======+==========+
| X(1) |   0.4500 |
+------+----------+
| X(2) |   0.3000 |
+------+----------+
| X(3) |   0.2500 |
+------+----------+


“phi(X)” is the probability value assigned to each outcome of X.  One thing to note is that these phi values don’t need to sum to one.  For example, I can multiple each probability value by one hundred as follows.

In [3]:
dist = DiscreteFactor(
    variables=["X"],
    cardinality=[3],
    values=[45, 30, 25],
    state_names= {'X': ['1', '2', '3']}
)

print(dist)

+------+----------+
| X    |   phi(X) |
+======+==========+
| X(1) |  45.0000 |
+------+----------+
| X(2) |  30.0000 |
+------+----------+
| X(3) |  25.0000 |
+------+----------+


pgmpy relaxes the constraint to sum to one is because the relaxation is useful in some algorithms.  We can always normalize to obtain proper probability values. 

In [4]:
# Normalize takes phi values for each outcome and divides them by their sum to get a probability. 
dist.normalize()

print(dist)

+------+----------+
| X    |   phi(X) |
+======+==========+
| X(1) |   0.4500 |
+------+----------+
| X(2) |   0.3000 |
+------+----------+
| X(3) |   0.2500 |
+------+----------+


## Modeling a joint distribution in pgmpy

In [5]:
joint = DiscreteFactor(
    variables=['X', 'Y'],   #A
    cardinality=[3, 2],    #B
    values=[.25, .20, .20, .10, .15, .10],   #C 
    state_names= {    
        'X': ['1', '2', '3'],    #C
        'Y': ['0', '1']    #C
    }
)

#A Now we have two variables instead of one.
#B X has 3 outcomes, Y has 2.
#C You can look at the printed output to see how the values are ordered of values.
#D Now there are two variables, so we name the outcomes for both variables.

print(joint)

+------+------+------------+
| X    | Y    |   phi(X,Y) |
+======+======+============+
| X(1) | Y(0) |     0.2500 |
+------+------+------------+
| X(1) | Y(1) |     0.2000 |
+------+------+------------+
| X(2) | Y(0) |     0.2000 |
+------+------+------------+
| X(2) | Y(1) |     0.1000 |
+------+------+------------+
| X(3) | Y(0) |     0.1500 |
+------+------+------------+
| X(3) | Y(1) |     0.1000 |
+------+------+------------+


The marginalize method will accomplish summing over the specified variables for us.


In [6]:
print(joint.marginalize(variables=['Y'], inplace=False))
print(joint.marginalize(variables=['X'], inplace=False))

+------+----------+
| X    |   phi(X) |
+======+==========+
| X(1) |   0.4500 |
+------+----------+
| X(2) |   0.3000 |
+------+----------+
| X(3) |   0.2500 |
+------+----------+
+------+----------+
| Y    |   phi(Y) |
+======+==========+
| Y(0) |   0.6000 |
+------+----------+
| Y(1) |   0.4000 |
+------+----------+


We can use a division operator on two factors to calculate conditional probabilities. Here, P(X, Y)/P(X) = P(Y|X).

In [7]:
print(joint / dist)

+------+------+------------+
| X    | Y    |   phi(X,Y) |
+======+======+============+
| X(1) | Y(0) |     0.5556 |
+------+------+------------+
| X(1) | Y(1) |     0.4444 |
+------+------+------------+
| X(2) | Y(0) |     0.6667 |
+------+------+------------+
| X(2) | Y(1) |     0.3333 |
+------+------+------------+
| X(3) | Y(0) |     0.6000 |
+------+------+------------+
| X(3) | Y(1) |     0.4000 |
+------+------+------------+


However, the more direct way in pgmpy to specify a conditional probability distribution table is with the TabularCPD class:

In [8]:
from pgmpy.factors.discrete.CPD import TabularCPD
PYgivenX = TabularCPD(
    variable='Y',    #A
    variable_card=2,    #B
    values=[
        [.25/.45, .20/.30, .15/.25],    #C
        [.20/.45, .10/.30, .10/.25],    #C
    ], 
    evidence=['X'],
    evidence_card=[3],
    state_names = {
        'X': ['1', '2', '3'],
        'Y': ['0', '1']
    })

#A Conditional distribution has one variable instead of DiscreteFactor’s list of variables.
#B variable_card is the cardinality of Y
#C Elements of the list correspond to outcomes for Y. Elements of each list correspond to elements of X.

print(PYgivenX)

+------+--------------------+---------------------+------+
| X    | X(1)               | X(2)                | X(3) |
+------+--------------------+---------------------+------+
| Y(0) | 0.5555555555555556 | 0.6666666666666667  | 0.6  |
+------+--------------------+---------------------+------+
| Y(1) | 0.4444444444444445 | 0.33333333333333337 | 0.4  |
+------+--------------------+---------------------+------+


## Canonical parameters in Pyro

In [9]:
import torch
from pyro.distributions import Bernoulli, Categorical, Gamma, Normal

# The categorical distribution takes a list of probability values, each value corresponding to an outcome.
print(Categorical(probs=torch.tensor([.45, .30, .25])))
print(Normal(loc=0.0, scale=1.0))
print(Bernoulli(probs=0.4))
print(Gamma(concentration=1.0, rate=2.0))

bern = Bernoulli(0.4)
lprob = bern.log_prob(torch.tensor(1.0))

import math
print(math.exp(lprob))


Categorical(probs: torch.Size([3]))
Normal(loc: 0.0, scale: 1.0)
Bernoulli(probs: 0.4000000059604645)
Gamma(concentration: 1.0, rate: 2.0)
0.3999999887335489


## Simulating from DiscreteFactor in pgmpy and Pyro

In [10]:
from pgmpy.factors.discrete import DiscreteFactor
dist = DiscreteFactor(
    variables=["X"],
    cardinality=[3],
    values=[.45, .30, .25],
    state_names= {'X': ['1', '2', '3']}
)

# n is the number of instances you wish to generate
dist.sample(n=3)


,X
0,1
1,1
2,2


In [11]:
joint = DiscreteFactor(
    variables=['X', 'Y'],
    cardinality=[3, 2],
    values=[.25, .20, .20, .10, .15, .10],
    state_names= {
        'X': ['1', '2', '3'],
        'Y': ['0', '1']
    }
)

joint.sample(n=3)


,X,Y
0,2,1
1,1,0
2,3,1


Canonical distributions in pyro also use a method with sample method.

In [12]:
import torch
from pyro.distributions import Categorical
# Generate 1 sample
one_sample = Categorical(probs=torch.tensor([.45, .30, .25])).sample()
print(one_sample)
# Generate 10 samples
samples = Categorical(probs=torch.tensor([.45, .30, .25])).sample(sample_shape=torch.Size([10]))
print(samples)

tensor(0)
tensor([0, 1, 2, 2, 1, 0, 2, 1, 1, 1])


## Creating a random process in pgmpy and pyro.

In [13]:
from pgmpy.factors.discrete.CPD import TabularCPD
from pgmpy.models import BayesianNetwork
from pgmpy.sampling import BayesianModelSampling

# Values is the probability value assigned to each outcome. The
PZ = TabularCPD(
    variable='Z', 
    variable_card=2,
    values=[[.65], [.35]],
    state_names = {
        'Z': ['0', '1']
    })

# The probability values map to the outcomes specified in the state_names argument.
PXgivenZ = TabularCPD(
    variable='X',
    variable_card=2,
    values=[
        [.8, .6],
        [.2, .4],
    ],
    evidence=['Z'],
    evidence_card=[2],
    state_names = {
        'X': ['0', '1'],
        'Z': ['0', '1']
    })

PYgivenX = TabularCPD(
    variable='Y',
    variable_card=3,
    values=[
        [.1, .8],
        [.2, .1],
        [.7, .1],
    ],
    evidence=['X'],
    evidence_card=[2],
    state_names = {
        'Y': ['1', '2', '3'],
        'X': ['0', '1']
    })

model = BayesianNetwork([('Z', 'X'), ('X', 'Y')])
model.add_cpds(PZ, PXgivenZ, PYgivenX)

generator = BayesianModelSampling(model)
generator.forward_sample(size=3)


  0%|          | 0/3 [00:00<?, ?it/s]

/Users/robertness/Dropbox/code/causalML/book/venv/lib/python3.11/site-packages/pgmpy/sampling/base.py:582: FutureWarning: Passing a DataFrame to DataFrame.from_records is deprecated. Use set_index and/or drop to modify the DataFrame instead.
  df = pd.DataFrame.from_records(samples)


,Z,X,Y
0,0,0,3
1,0,0,3
2,0,0,2


# Working with combinations of canonical distributions in Pyro

In [14]:
import torch
from pyro.distributions import Bernoulli, Poisson, Gamma

# Represent P(Z) with a Gamma distribution, and sample z.
z = Gamma(7.5, 1.0).sample()    #A
# Represent P(X|Z=z) with a Poisson distribution with location parameter z, and sample x.
x = Poisson(z).sample()    #B
# Represent P(Y|X=x) with a Bernoulli distribution. The probability parameter is a function of y.
y = Bernoulli(x / (5+x)).sample()    #C

print(z, x, y)


tensor(7.2970) tensor(6.) tensor(0.)


# Random processes with nuanced control flow in Pyro

In [15]:
import torch
from pyro.distributions import Bernoulli, Poisson, Gamma
z = Gamma(7.5, 1.0).sample()
x = Poisson(z).sample()
# y is calculated as a sum of random coin flips.
# y is generated from P(Y|X=x) because the number of flips depends on x.
y = torch.tensor(0.0)
for i in range(int(x)):
    y += Bernoulli(.5).sample()    
print(z, x, y)

tensor(8.5513) tensor(10.) tensor(6.)


# Using functions for random processes and pyro.sample

In the code below, we generate multiple samples and then use the `mean` method to calculate an average of those samples.

In [16]:
import torch
import pyro
def random_process():
    z = pyro.sample("z", Gamma(7.5, 1.0))
    x = pyro.sample("x", Poisson(z))
    y = torch.tensor(0.0)
    for i in range(int(x)):
        y += pyro.sample(f"y{i}", Bernoulli(.5))    #A
    return y

generated_samples = generator.forward_sample(size=100)
generated_samples['Y'].apply(int).mean()


  0%|          | 0/3 [00:00<?, ?it/s]

/Users/robertness/Dropbox/code/causalML/book/venv/lib/python3.11/site-packages/pgmpy/sampling/base.py:582: FutureWarning: Passing a DataFrame to DataFrame.from_records is deprecated. Use set_index and/or drop to modify the DataFrame instead.
  df = pd.DataFrame.from_records(samples)


2.2

In [17]:
generated_samples_ = torch.stack([random_process() for _ in range(100)])
generated_samples_.mean()

tensor(3.9900)

In [18]:
torch.square(generated_samples_).mean()

tensor(22.8900)

# Monte Carlo estimation in Pyro

In [19]:
import torch
import pyro
from pyro.distributions import Bernoulli, Gamma, Poisson
# This new version of random_process now returns both z and y.
def random_process():
    z = pyro.sample("z", Gamma(7.5, 1.0))
    x = pyro.sample("x", Poisson(z))
    y = torch.tensor(0.0)
    for i in range(int(x)):
        y += pyro.sample(f"{i}", Bernoulli(.5))
    return z, y

# Generate 1000 instances of z and y using a list comprehension.
generated_samples = [random_process() for _ in range(1000)]
# Turn the individual z tensors into a single tensor,
# then calculate the Monte Carlo estimate via the mean method.
z_mean = torch.stack([z for z, _ in generated_samples]).mean()

print(z_mean)

z_given_y = torch.stack([z for z, y in generated_samples if y == 3])

print(z_given_y.mean())


tensor(7.5682)
tensor(7.1242)


In [20]:
generator.forward_sample(size=10)

  0%|          | 0/3 [00:00<?, ?it/s]

/Users/robertness/Dropbox/code/causalML/book/venv/lib/python3.11/site-packages/pgmpy/sampling/base.py:582: FutureWarning: Passing a DataFrame to DataFrame.from_records is deprecated. Use set_index and/or drop to modify the DataFrame instead.
  df = pd.DataFrame.from_records(samples)


,Z,X,Y
0,0,0,3
1,0,0,3
2,0,0,3
3,1,1,1
4,0,0,3
5,0,0,3
6,0,0,3
7,1,0,1
8,1,0,2
9,1,0,3


# Generating IID samples in Pyro

In [21]:
import pyro
from pyro.distributions import Bernoulli, Poisson, Gamma

def model():
    z = pyro.sample("z", Gamma(7.5, 1.0))
    x = pyro.sample("x", Poisson(z))
    # pyro.plate is a context manager for generating conditional independent
    # samples. This instance of pyro.plate will generate 10 IID samples.
    with pyro.plate('IID', 10):
        # Calling pyro.sample to generates a single outcome y, where y is a
        # tensor of 10 IID samples.
        y = pyro.sample("y", Bernoulli(x / (5+x)))
    return y

model()

tensor([1., 0., 1., 0., 0., 0., 0., 1., 1., 0.])

# An example of a data generating process in code form

In [22]:
def true_dgp(jenny_inclination, brian_inclination, window_strength):    #A
    jenny_throws_rock = jenny_inclination > 0.5    #B
    brian_throws_rock = brian_inclination > 0.5    #B
    if jenny_throws_rock and brian_throws_rock:    #C
        strength_of_impact = 0.8    #C
    elif jenny_throws_rock or brian_throws_rock:    #D
        strength_of_impact = 0.6    #D
    else:    #E
        strength_of_impact = 0.0    #E
    window_breaks = window_strength < strength_of_impact    #F
    return jenny_throws_rock, brian_throws_rock, window_breaks

#A Input variables reflect Jenny and Brian’s inclination to throw and the window strength.
#B Jenny and Brian throw the rock if so inclined.
#C If both Jenny and Brian throw the rock, the total strength of the impact is .8.
#D If either Jenny or Brian throws the rock, the total strength of the impact is .6.
#E Otherwise, no one throws and the strength of impact is 0.
#F If the strength of impact is greater than the strength of the window, the window breaks.

initials = [
    (0.6, 0.31, 0.83),
    (0.48, 0.53, 0.33),
    (0.66, 0.63, 0.75),
    (0.65, 0.66, 0.8),
    (0.48, 0.16, 0.27)
]

data_points = []
# Now we pass each tuple in the initials list as an input to the true_dgp function, generate a data point, and add this to the data_points list. 
for jenny_inclination, brian_inclination, window_strength in initials:    
    data_points.append(
        true_dgp(
            jenny_inclination, brian_inclination, window_strength
        )
    )

print(data_points)

[(True, False, False), (False, True, True), (True, True, True), (True, True, False), (False, False, False)]
